In [1]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../../data/'
group_dir_ = '../../data/groups/high'
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'

In [2]:
all_user = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
predictions = pd.read_pickle(os.path.join(dir_, 'post_filtering_old', str(0.1) + '_prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))

In [3]:
predictions[:5]

,uid,tid,rating
4,0,31,2.377213
6,0,37,2.094848
10,0,72,2.319738
13,0,90,2.195360
18,0,122,2.344907


In [4]:
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,rating,count
7,0,3,0.25,1
11,0,7,2.50,3
32,0,18,1.50,2
34,0,20,2.50,3
45,0,31,5.00,17


In [5]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10', '20']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [6]:
num_user = len(predictions['uid'].unique())
num_user

220

In [7]:
pop_dict = {}
for i in all_user['tid'].unique():
    pop_dict[i] = len(all_user[all_user['tid']==i])/num_user

In [8]:
pop_4 = []
pop_20 = []
pop_50 = []

In [9]:
top_n_size = 50
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([50, 20, 4]):
    avg_pops = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        pops =[]
        for j in (range(len(groups))):
            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    tid = top_n_list[l]
                    pops.append(pop_dict[tid])
        avg_pop = sum(pops)/len(pops)
        avg_pops.append(avg_pop)
        
    if top_n_size == 4:
        pop_4.append(avg_pops)
    elif top_n_size == 20:
        pop_20.append(avg_pops)
    elif top_n_size == 50:
        pop_50.append(avg_pops)

In [10]:
for i in pop_4:
    print(i)
for i in pop_20:
    print(i)
for i in pop_50:
    print(i)

[0.4434400826446277, 0.38711363636363655, 0.34244868035190584, 0.30730519480519475, 0.2764914772727272, 0.2600524475524481, 0.2450000000000005, 0.20880681818181823, 0.20355113636363656, 0.12045454545454529]
[0.3517613636363622, 0.31245454545454204, 0.2734090909090898, 0.25297619047618863, 0.22436079545454476, 0.20683566433566486, 0.19113636363636424, 0.166619318181818, 0.18198863636363627, 0.08988636363636396]
[0.30533367768595193, 0.2737345454545481, 0.23990615835777077, 0.2213636363636338, 0.19807954545454487, 0.17896503496503527, 0.16430000000000008, 0.14755681818181787, 0.15673863636363602, 0.08281818181818167]
